<a href="https://colab.research.google.com/github/rishikeshvk/Assessment_12-06-24_RishikeshVK/blob/main/rishikesh_12_06_24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print("Hello World")

Hello World
